In [2]:
import xarray as xr
import numpy as np
# import statsmodels.api as sm
import pandas as pd
import dask.dataframe as dd
from copy import deepcopy
from dask.distributed import Client, LocalCluster
from dask_jobqueue import PBSCluster
import dask
from matplotlib.colors import LogNorm
import matplotlib.pyplot as plt
from collections import Counter
from cartopy import crs as ccrs
from scipy.stats import percentileofscore
import cartopy.feature as cfeature
import warnings
warnings.simplefilter("ignore")
import glob
%matplotlib inline
import os

In [3]:
files = []
base_path = "/glade/scratch/cbecker/ptype_case_studies/kentucky/rap/"
for date in os.listdir(base_path):
    for init_time in os.listdir(os.path.join(base_path, date)):
        sub_files = os.listdir(os.path.join(base_path, date, init_time))
        for f in sub_files:
            files.append(os.path.join(base_path, date, init_time, f))
files = sorted(files)

In [22]:
cluster = PBSCluster(account='NAML0001',
                     queue='casper',
                     walltime='01:00:00',
                     memory="200 GB",
                     cores=24,
                     interface='ib0')

# Change your url to the dask dashboard so you can see it
dask.config.set({'distributed.dashboard.link':'https://jupyterhub.hpc.ucar.edu/stable/user/{USER}/proxy/{port}/status'})
                    
client = Client(cluster)
cluster.scale(jobs=10, memory="200GB")

In [23]:
cluster

Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/cbecker/proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.12.206.57:33896,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/cbecker/proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [ ]:
%%time
ds = xr.open_mfdataset(files[:400], parallel=True, engine='netcdf4', decode_cf=False, concat_dim='time', combine='nested', 
                       chunks={'time':1, 'heightAboveGround': 21, 'y':337, 'x': 451, 'isobaricInhPa': 37}).persist()

In [21]:
xr.open_dataset(files[4])

<xarray.Dataset>
Dimensions:                  (time: 1, heightAboveGround: 21, y: 337, x: 451,
                              isobaricInhPa: 37)
Coordinates:
  * heightAboveGround        (heightAboveGround) int64 0 250 500 ... 4750 5000
  * time                     (time) datetime64[ns] 2022-02-23
    step                     timedelta64[ns] ...
  * isobaricInhPa            (isobaricInhPa) float32 1e+03 975.0 ... 125.0 100.0
    latitude                 (y, x) float32 ...
    longitude                (y, x) float32 ...
    valid_time               datetime64[ns] ...
    y_projection_coordinate  (y) float32 ...
    x_projection_coordinate  (x) float32 ...
Dimensions without coordinates: y, x
Data variables: (12/29)
    t_h                      (time, heightAboveGround, y, x) float32 ...
    dpt_h                    (time, heightAboveGround, y, x) float32 ...
    u_h                      (time, heightAboveGround, y, x) float32 ...
    v_h                      (time, heightAboveGround, y, x) float32 ...
    isobaricInhPa_h          (time, heightAboveGround, y, x) float64 ...
    t                        (time, isobaricInhPa, y, x) float32 ...
    ...                       ...
    ML_snow                  (time, y, x) float32 ...
    ML_csnow                 (time, y, x) uint8 ...
    ML_icep                  (time, y, x) float32 ...
    ML_cicep                 (time, y, x) uint8 ...
    ML_frzr                  (time, y, x) float32 ...
    ML_cfrzr                 (time, y, x) uint8 ...